# Tags Example

This example demonstrates the **tags feature** in PyProforma v2.

Tags provide a flexible way to categorize line items and sum them by tag in formulas. Unlike hierarchical categories, tags enable multi-dimensional grouping and analysis.

In this example, we'll use tags to:
- Categorize line items by **type** (income vs. expense)
- Distinguish between **operational** and **non-operational** items
- Identify **recurring** vs. **non-recurring** items

## Import Required Libraries

In [1]:
from pyproforma.v2 import FixedLine, FormulaLine, ProformaModel

## Define Financial Model with Tags

We'll create a financial model that uses tags to group line items along three dimensions:

1. **Type**: `income` vs. `expense` vs. `balance_sheet`
2. **Nature**: `operational` vs. `non-operational`
3. **Reporting**: `recurring` vs. `non-recurring`

Each line item can have multiple tags, enabling flexible multi-dimensional categorization and analysis.

In [2]:
class FinancialStatementWithTags(ProformaModel):
    """Financial model demonstrating tag-based categorization."""

    # Revenue items - all tagged as income and operational
    product_revenue = FixedLine(
        values={2024: 1000, 2025: 1100, 2026: 1210},
        label="Product Revenue",
        tags=["income", "operational", "recurring"],
    )

    service_revenue = FixedLine(
        values={2024: 500, 2025: 550, 2026: 605},
        label="Service Revenue",
        tags=["income", "operational", "recurring"],
    )

    # One-time income
    asset_sale_gain = FixedLine(
        values={2024: 0, 2025: 150, 2026: 0},
        label="Gain on Asset Sale",
        tags=["income", "non-operational", "non-recurring"],
    )

    # Interest income
    interest_income = FixedLine(
        values={2024: 20, 2025: 25, 2026: 30},
        label="Interest Income",
        tags=["income", "non-operational", "recurring"],
    )

    # Operating expenses
    cost_of_goods_sold = FixedLine(
        values={2024: 600, 2025: 660, 2026: 726},
        label="Cost of Goods Sold",
        tags=["expense", "operational", "recurring"],
    )

    salaries = FixedLine(
        values={2024: 400, 2025: 420, 2026: 441},
        label="Salaries & Wages",
        tags=["expense", "operational", "recurring"],
    )

    marketing = FixedLine(
        values={2024: 150, 2025: 165, 2026: 182},
        label="Marketing",
        tags=["expense", "operational", "recurring"],
    )

    # Non-operating expenses
    interest_expense = FixedLine(
        values={2024: 50, 2025: 45, 2026: 40},
        label="Interest Expense",
        tags=["expense", "non-operational", "recurring"],
    )

    restructuring_costs = FixedLine(
        values={2024: 100, 2025: 0, 2026: 0},
        label="Restructuring Costs",
        tags=["expense", "non-operational", "non-recurring"],
    )

    # Calculated totals using tags
    total_income = FormulaLine(
        formula=lambda a, li, t: li.tag["income"][t], label="Total Income"
    )

    total_expenses = FormulaLine(
        formula=lambda a, li, t: li.tag["expense"][t], label="Total Expenses"
    )

    # Operating profit (operational items only)
    operating_profit = FormulaLine(
        formula=lambda a, li, t: (
            li.product_revenue[t]
            + li.service_revenue[t]
            - li.cost_of_goods_sold[t]
            - li.salaries[t]
            - li.marketing[t]
        ),
        label="Operating Profit",
    )

    # Net income
    net_income = FormulaLine(
        formula=lambda a, li, t: li.total_income[t] - li.total_expenses[t],
        label="Net Income",
    )

    # Recurring income (excludes one-time items)
    recurring_income = FormulaLine(
        formula=lambda a, li, t: (
            li.product_revenue[t]
            + li.service_revenue[t]
            + li.interest_income[t]
            - li.cost_of_goods_sold[t]
            - li.salaries[t]
            - li.marketing[t]
            - li.interest_expense[t]
        ),
        label="Recurring Net Income",
        tags=["calculated", "recurring"],
    )

## Create Model Instance

Let's instantiate the model with three periods: 2024, 2025, and 2026.

In [3]:
# Create model
model = FinancialStatementWithTags(periods=[2024, 2025, 2026])

## Access Tags via LineItemResult

Each line item result has a `tags` property that returns the list of tags associated with that line item.

In [4]:
print("Line Item Tags:")
print("-" * 80)
revenue = model["product_revenue"]
print(f"Product Revenue tags: {revenue.tags}")

expenses = model["cost_of_goods_sold"]
print(f"COGS tags: {expenses.tags}")

Line Item Tags:
--------------------------------------------------------------------------------
Product Revenue tags: ['income', 'operational', 'recurring']
COGS tags: ['expense', 'operational', 'recurring']


## Tag-Based Summations

Use the `model.li.tag['tag_name'][period]` syntax to sum all line items with a specific tag.

This powerful feature enables dynamic calculations across multiple dimensions without explicitly listing each line item.

In [5]:
print("Tag-Based Summations (2024):")
print("-" * 80)
print(
    f"Total income (all 'income' tagged items):     ${model.li.tag['income'][2024]:>10,.0f}"
)
print(
    f"Total expenses (all 'expense' tagged items):  ${model.li.tag['expense'][2024]:>10,.0f}"
)
print(
    f"Operational items sum:                        ${model.li.tag['operational'][2024]:>10,.0f}"
)
print(
    f"Non-operational items sum:                    ${model.li.tag['non-operational'][2024]:>10,.0f}"
)
print(
    f"Recurring items sum:                          ${model.li.tag['recurring'][2024]:>10,.0f}"
)
print(
    f"Non-recurring items sum:                      ${model.li.tag['non-recurring'][2024]:>10,.0f}"
)

Tag-Based Summations (2024):
--------------------------------------------------------------------------------
Total income (all 'income' tagged items):     $     1,520
Total expenses (all 'expense' tagged items):  $     1,300
Operational items sum:                        $     2,650
Non-operational items sum:                    $       170
Recurring items sum:                          $     3,040
Non-recurring items sum:                      $       100


## Generate Income Statements

Let's create formatted income statements for all periods, demonstrating how tags work across multiple years.

In [6]:
# Show income statement for each year
for year in model.periods:
    print(f"\nIncome Statement - {year}:")
    print("-" * 80)
    print(f"Product Revenue          ${model.li.product_revenue[year]:>12,.0f}")
    print(f"Service Revenue          ${model.li.service_revenue[year]:>12,.0f}")
    print(f"Asset Sale Gain          ${model.li.asset_sale_gain[year]:>12,.0f}")
    print(f"Interest Income          ${model.li.interest_income[year]:>12,.0f}")
    print(f"                         {'-' * 14}")
    print(f"Total Income             ${model.li.total_income[year]:>12,.0f}")
    print()
    print(f"Cost of Goods Sold       ${model.li.cost_of_goods_sold[year]:>12,.0f}")
    print(f"Salaries & Wages         ${model.li.salaries[year]:>12,.0f}")
    print(f"Marketing                ${model.li.marketing[year]:>12,.0f}")
    print(f"Interest Expense         ${model.li.interest_expense[year]:>12,.0f}")
    print(f"Restructuring Costs      ${model.li.restructuring_costs[year]:>12,.0f}")
    print(f"                         {'-' * 14}")
    print(f"Total Expenses           ${model.li.total_expenses[year]:>12,.0f}")
    print()
    print(f"                         {'=' * 14}")
    print(f"Net Income               ${model.li.net_income[year]:>12,.0f}")
    print(f"                         {'=' * 14}")
    print()
    print(f"Recurring Net Income     ${model.li.recurring_income[year]:>12,.0f}")


Income Statement - 2024:
--------------------------------------------------------------------------------
Product Revenue          $       1,000
Service Revenue          $         500
Asset Sale Gain          $           0
Interest Income          $          20
                         --------------
Total Income             $       1,520

Cost of Goods Sold       $         600
Salaries & Wages         $         400
Marketing                $         150
Interest Expense         $          50
Restructuring Costs      $         100
                         --------------
Total Expenses           $       1,300

Net Income               $         220

Recurring Net Income     $         320

Income Statement - 2025:
--------------------------------------------------------------------------------
Product Revenue          $       1,100
Service Revenue          $         550
Asset Sale Gain          $         150
Interest Income          $          25
                         --------------


## Key Features Demonstrated

This example showcased the following tag capabilities:

1. **Multiple tags per line item** - Enable flexible categorization along multiple dimensions
2. **Tag access via `LineItemResult.tags`** - Retrieve tags for any line item result
3. **Tag-based summation** - Use `li.tag['tag_name'][period]` to sum all items with a specific tag
4. **Tags in formulas** - Reference tag-based sums directly in `FormulaLine` calculations
5. **Tag-based selection** - Use `model.tag['tag_name']` to get a `LineItemSelection` of all items with a tag
6. **Selection operations** - Get values (`.value()`) and generate tables (`.table()`) for tagged items
7. **Multi-dimensional analysis** - Analyze by operational status, recurring nature, and more

Tags provide a powerful alternative to hierarchical categories when you need flexible, multi-dimensional grouping of line items.

## Select Line Items by Tag

You can also use `model.tag['tag_name']` to get a **LineItemSelection** containing all line items with a specific tag. This returns a selection object that you can use to:

- Access the list of matching line items
- Get values for all selected items at once
- Generate tables with only the tagged items

In [7]:
# Select all line items with "income" tag
income_selection = model.tag["income"]

print("Line items with 'income' tag:")
print(income_selection.names)
print()

# Select all operational items
operational_selection = model.tag["operational"]

print("Line items with 'operational' tag:")
print(operational_selection.names)

Line items with 'income' tag:
['product_revenue', 'service_revenue', 'asset_sale_gain', 'interest_income']

Line items with 'operational' tag:
['product_revenue', 'service_revenue', 'cost_of_goods_sold', 'salaries', 'marketing']


### Get Values by Tag

The selection's `.value()` method returns a dictionary of all selected line items and their values for a specific period:

In [8]:
# Get values for all income items in 2024
income_values_2024 = income_selection.value(2024)

print("Income items for 2024:")
print("-" * 80)
for item_name, value in income_values_2024.items():
    print(f"{item_name:30} ${value:>12,.0f}")
print("-" * 80)
print(f"{'Total':30} ${sum(income_values_2024.values()):>12,.0f}")

Income items for 2024:
--------------------------------------------------------------------------------
product_revenue                $       1,000
service_revenue                $         500
asset_sale_gain                $           0
interest_income                $          20
--------------------------------------------------------------------------------
Total                          $       1,520


### Generate Tables by Tag

You can generate formatted tables containing only the line items with a specific tag. This is perfect for creating focused reports like "All Expenses" or "Recurring Income Items":

In [9]:
# Generate a table of all income items
income_table = model.tag["income"].table()

print("Income Items Table:")
income_table.show()
print()

# Generate a table of all expense items
expense_table = model.tag["expense"].table()

print("Expense Items Table:")
expense_table.show()

Income Items Table:


Name,2024,2025,2026
product_revenue,"1,000","1,100","1,210"
service_revenue,500,550,605
asset_sale_gain,0,150,0
interest_income,20,25,30
Total,"1,520","1,825","1,845"



Expense Items Table:


Name,2024,2025,2026
cost_of_goods_sold,600,660,726
salaries,400,420,441
marketing,150,165,182
interest_expense,50,45,40
restructuring_costs,100,0,0
Total,"1,300","1,290","1,389"


In [10]:
# Generate a table of recurring items (excluding one-time items)
recurring_table = model.tag["recurring"].table(include_label=True)

print("Recurring Items Table:")
recurring_table.show()

Recurring Items Table:


Name,Label,2024,2025,2026
product_revenue,Product Revenue,"1,000","1,100","1,210"
service_revenue,Service Revenue,500,550,605
interest_income,Interest Income,20,25,30
cost_of_goods_sold,Cost of Goods Sold,600,660,726
salaries,Salaries & Wages,400,420,441
marketing,Marketing,150,165,182
interest_expense,Interest Expense,50,45,40
recurring_income,Recurring Net Income,320,385,456
,Total,"3,040","3,350","3,690"
